<a href="https://colab.research.google.com/github/rubanzasilva/flood_probability/blob/dev/s04e05_fastai_exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression with a Flood Prediction Dataset

Playground Series - Season 4, Episode 5 where we are tasked with predicting the likelihood of floods in certain areas based off various factors.

In this notebook, i have used  AutoGluon, which is an automated machine learning framework that promises to help you build fast and accurate ML models in three lines of code.

I shall put that to the test and compare it with my more manual process from my previous notebook located [here](https://www.kaggle.com/code/rubanzasilva/s04e05-fastai).

In [ ]:
!pip install autogluon

In [2]:
#hide
#! [ -e /content ]

#hide
#This imports and sets up everything you will need for this notebook
#
#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

#from fastbook import *
#!pip install ucimlrepo
#from ucimlrepo import fetch_ucirepo

from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
#from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
from xgboost import plot_importance

import lightgbm as lgb

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv

from ipywidgets import interact


matplotlib.rc('image', cmap='Greys')

#from fastkaggle import setup_comp
import optuna
#import optuna_distributed

from autogluon.tabular import TabularDataset, TabularPredictor

/opt/conda/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv
/kaggle/input/flood-prediction-factors/flood.csv


In [4]:
!ls /kaggle/input/playground-series-s4e5

sample_submission.csv  test.csv  train.csv


In [5]:
!ls /kaggle/input/flood-prediction-factors 

flood.csv


In [6]:
#set_seed(42)

In [7]:
path = Path('/kaggle/input/playground-series-s4e5/')
path

Path('/kaggle/input/playground-series-s4e5')

In [8]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv',index_col='id')
original_df = pd.read_csv('/kaggle/input/flood-prediction-factors/flood.csv')

In [9]:
original_df.shape,train_df.shape

((50000, 21), (1117957, 21))

In [10]:
train_df = pd.concat([train_df,original_df], axis=0)
train_df.shape

(1167957, 21)

In [11]:
train_df.shape,test_df.shape

((1167957, 21), (745305, 20))

In [12]:
#train_df.hist(figsize=(20,15));

# AutoGluon Train model

In [13]:
%%time

predictor = TabularPredictor(label='FloodProbability').fit(train_data=train_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20240515_023004"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240515_023004"
=================== System

[1000]	valid_set's rmse: 0.0199416
[2000]	valid_set's rmse: 0.0198346
[3000]	valid_set's rmse: 0.0197893
[4000]	valid_set's rmse: 0.0197668
[5000]	valid_set's rmse: 0.0197485
[6000]	valid_set's rmse: 0.0197356
[7000]	valid_set's rmse: 0.0197272
[8000]	valid_set's rmse: 0.0197224
[9000]	valid_set's rmse: 0.0197191
[10000]	valid_set's rmse: 0.019711


	-0.0197	 = Validation score   (-root_mean_squared_error)
	818.92s	 = Training   runtime
	8.15s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 0.0198579
[2000]	valid_set's rmse: 0.0197987
[3000]	valid_set's rmse: 0.0197629
[4000]	valid_set's rmse: 0.0197453
[5000]	valid_set's rmse: 0.0197325
[6000]	valid_set's rmse: 0.0197309
[7000]	valid_set's rmse: 0.019733
[8000]	valid_set's rmse: 0.0197366


	-0.0197	 = Validation score   (-root_mean_squared_error)
	626.64s	 = Training   runtime
	2.67s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.0342	 = Validation score   (-root_mean_squared_error)
	1140.27s	 = Training   runtime
	0.48s	 = Validation runtime
Fitting model: CatBoost ...
	-0.0195	 = Validation score   (-root_mean_squared_error)
	2130.75s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.0341	 = Validation score   (-root_mean_squared_error)
	652.8s	 = Training   runtime
	0.7s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.0188	 = Validation score   (-root_mean_squared_error)
	929.53s	 = Training   runtime
	0.17s	 = Validation runtime
Fitting model: XGBoost ...
	-0.0201	 = Validation score   (-root_mean_squared_error)
	625.89s	 = Training   runtime
	4.68s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-0.0194	 = Validation score   (-root_mean_squared_error)
	464.66s	 = Training   runtime
	0.05s	 = V

[1000]	valid_set's rmse: 0.019948
[2000]	valid_set's rmse: 0.0199006
[3000]	valid_set's rmse: 0.0198917
[4000]	valid_set's rmse: 0.0198945


	-0.0199	 = Validation score   (-root_mean_squared_error)
	429.95s	 = Training   runtime
	2.59s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 0.917, 'NeuralNetTorch': 0.083}
	-0.0188	 = Validation score   (-root_mean_squared_error)
	0.05s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 7879.51s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240515_023004")


CPU times: user 5h 5min 27s, sys: 1min 12s, total: 5h 6min 39s
Wall time: 2h 11min 20s


In [14]:
predictions = predictor.predict(test_df)

In [15]:
predictions.shape

(745305,)

In [16]:
predictor.evaluate(train_df)

{'root_mean_squared_error': -0.01888253524043939,
 'mean_squared_error': -0.0003565501371064355,
 'mean_absolute_error': -0.014848477698917565,
 'r2': 0.8628821830957755,
 'pearsonr': 0.9289665627881948,
 'median_absolute_error': -0.01187482595443723}

In [ ]:
#predictor.leaderboard(train_df)

In [ ]:
test_df['Rings'] = predictions
test_df.to_csv('submission.csv', columns=['Rings'], index=True)

submission = pd.read_csv('submission.csv')
submission.head()

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "auto gluom  "

In [ ]:
#0.05 no original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
learn.export('models/fp_colab_model.pkl')

In [ ]:
learn.load('fp_colab_model.pkl')

# Model Ensemble